<a href="https://colab.research.google.com/github/prinzessinmarlenifee/SenseCap/blob/main/Data_Augmentation_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vergleich Data Augmentation Factors

###Inhalt

Mehrere Modelle mit verschiedenen Augmentierungsgraden (augment_factor) vergleichen

Modelle nicht immer neu trainieren – lieber einmal speichern/laden

Nicht nur Accuracy, sondern gezielt Precision & Recall für "Action" beobachten

Eine schöne 📊 Vergleichsgrafik zum Augmentierungsgrad


📦 Zusammengefasst:
Teil	Was du brauchst

📁 Modell-Dateien	 -  global_aug_factor_0.keras, global_aug_factor_1.keras, ...


📄 Validierungsdaten - 	X_val.npy, y_val.npy


🔡 LabelEncoder	label_encoder.pkl (oder neu erstellen bei gleichem Dataset)

#🧪 Schritt 1: Setup

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model
import joblib  # falls du LabelEncoder gespeichert hast


#📂 Schritt 2: Pfade & Klassen setzen

In [ ]:
# Anpassen auf dein Setup
model_dir = "/content/drive/MyDrive/mtb_project/final_models_global"
X_val_path = "/content/drive/MyDrive/mtb_project/X_val.npy"
y_val_path = "/content/drive/MyDrive/mtb_project/y_val.npy"
le_path = "/content/drive/MyDrive/mtb_project/label_encoder.pkl"  # wenn gespeichert, nach v4.2 f3 passiert

# Laden
X_val = np.load(X_val_path)
y_val = np.load(y_val_path)
le = joblib.load(le_path)


#🔍 Schritt 3: Analysefunktion

In [ ]:
def evaluate_model(model_path, X_val, y_val, le, target_class="Action"):
    model = load_model(model_path)
    y_pred_probs = model.predict(X_val, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)

    report = classification_report(
        y_val, y_pred,
        target_names=le.classes_,
        output_dict=True
    )

    # Optional anzeigen
    print(f"\n📄 Report für {os.path.basename(model_path)}")
    print(classification_report(y_val, y_pred, target_names=le.classes_))

    # Werte extrahieren
    precision = report[target_class]["precision"]
    recall = report[target_class]["recall"]
    f1 = report[target_class]["f1-score"]
    acc = report["accuracy"]

    return precision, recall, f1, acc


#🔁 Schritt 4: Alle Modelle evaluieren

In [ ]:
augment_factors = [0, 1, 2, 3]  # oder mehr
metrics_dict = {}

for factor in augment_factors:
    model_path = os.path.join(model_dir, f"global_aug_factor_{factor}.keras")
    precision, recall, f1, acc = evaluate_model(model_path, X_val, y_val, le, target_class="Action")
    metrics_dict[factor] = {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": acc
    }


#📊 Schritt 5: Plotten

In [ ]:
def plot_metrics(metrics_dict):
    factors = sorted(metrics_dict.keys())
    precision = [metrics_dict[f]["precision"] for f in factors]
    recall = [metrics_dict[f]["recall"] for f in factors]
    f1 = [metrics_dict[f]["f1"] for f in factors]
    acc = [metrics_dict[f]["accuracy"] for f in factors]

    plt.figure(figsize=(10, 6))
    plt.plot(factors, precision, label="Precision (Action)", marker="o")
    plt.plot(factors, recall, label="Recall (Action)", marker="o")
    plt.plot(factors, f1, label="F1-Score (Action)", marker="o")
    plt.plot(factors, acc, label="Accuracy (Overall)", linestyle="--", marker="x")
    plt.title("📈 Einfluss von Augmentierungsgrad auf Action-Klassifikation")
    plt.xlabel("Augmentierungsfaktor")
    plt.ylabel("Score")
    plt.ylim(0, 1.05)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# 🎯 Plot starten
plot_metrics(metrics_dict)
